In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("../data/data_imputed_2.csv")

In [3]:
df.head()

,timestamp_local,temp,city_name,country_code,aqi,co,no2,o3,pm10,pm25,so2
0,01/02/2022 0:00,12.6,Gujrāt,PK,385,1339.8,76.0,10.7,491.7,347.67,238.0
1,01/02/2022 1:00,11.5,Gujrāt,PK,404,1437.6,76.0,9.3,508.3,359.33,268.0
2,01/02/2022 2:00,11.9,Gujrāt,PK,421,1535.5,76.0,8.0,525.0,371.00,298.0
3,01/02/2022 3:00,12.2,Gujrāt,PK,425,1659.0,68.3,5.3,529.3,374.00,275.7
4,01/02/2022 4:00,11.9,Gujrāt,PK,430,1782.5,60.7,2.7,533.7,377.00,253.3


In [4]:
from sklearn.model_selection import train_test_split

In [5]:
data = df[['no2']]

In [6]:
from sklearn.preprocessing import MinMaxScaler
values = data.values

values = values.astype('float32')

scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)

In [7]:
scaled = scaled.reshape((19706,))

In [8]:
scaled.shape

(19706,)

In [9]:
def df_to_X_y(df, window_size, forecast_length):
    # df_as_np = df.to_numpy()
    X = []
    y = []
    
    for i in range(len(df) - window_size - forecast_length):
        row_x = [[a] for a in df[i:i+window_size]]
        
        X.append(row_x)
        # row_y = df_as_np[i+window_size]
        row_y = [b for b in df[i+window_size:i+window_size+forecast_length]]
        
        y.append(row_y)
        
    return np.array(X), np.array(y)

In [10]:
WINDOW_WIDTH = 168
FORECAST_LENGTH = 168
X, y = df_to_X_y(scaled, WINDOW_WIDTH, FORECAST_LENGTH)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

X_max = X_train.max()
X_min = X_train.min()

In [11]:
from keras.models import Sequential, save_model, load_model
from keras.layers import *
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from keras.losses import MeanSquaredError
from keras.metrics import RootMeanSquaredError

In [12]:
normalizer = Normalization()
normalizer.adapt(X_train)
rlrop = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10)

model = Sequential()
# 1D Convolutional layers
model.add(Conv1D(filters=8, kernel_size=3, activation='relu', input_shape=X_train.shape[1:]))
# model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
# model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
# Flatten the output of the Conv layers
model.add(Flatten())
# model.add(LSTM(input_shape = X_train.shape[1:], units=86, return_sequences=True))
# model.add(LSTM(units=64, return_sequences=True))
# model.add(GRU(units=86, return_sequences=True))
# model.add(LSTM(units=128, return_sequences=True))
# model.add(GRU(units=100, return_sequences=True))
# model.add(LSTM(units=100, return_sequences=False))
# model.add(Dense(units=100,  activation='relu'))
# model.add(Dense(8, activation='relu'))

model.add(Dense(units=FORECAST_LENGTH, activation='linear'))

model.summary()

model.compile(optimizer=Adam(learning_rate=0.001), metrics=[RootMeanSquaredError()] , loss=MeanSquaredError())

history = model.fit(X_train, y_train, validation_data=(X_val,y_val), epochs=50, callbacks=[rlrop])

# score = model.evaluate(X_test, y_test, verbose = 1)

C:\Users\Hassan\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                      │ (None, 166, 8)              │              32 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 1328)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 168)                 │         223,272 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 223,304 (872.28 KB)

 Trainable params: 223,304 (872.28 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
388/388 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 0.0169 - root_mean_squared_error: 0.1285 - val_loss: 0.0119 - val_root_mean_squared_error: 0.1093 - learning_rate: 0.0010
Epoch 2/50
388/388 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0119 - root_mean_squared_error: 0.1092 - val_loss: 0.0120 - val_root_mean_squared_error: 0.1093 - learning_rate: 0.0010
Epoch 3/50
388/388 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0115 - root_mean_squared_error: 0.1073 - val_loss: 0.0117 - val_root_mean_squared_error: 0.1083 - learning_rate: 0.0010
Epoch 4/50
388/388 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - loss: 0.0115 - root_mean_squared_error: 0.1071 - val_loss: 0.0117 - val_root_mean_squared_error: 0.1079 - learning_rate: 0.0010
Epoch 5/50
388/388 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - loss: 0.0115 - root_mean_squared_error: 0.1073 - val_loss: 0.0116 - val_root_mean_squared_error: 0.1078 - learning_rate: 0.0010
Epoch 6/50
388/388 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 0.0112 - root_mean_squared_err

In [13]:
forecast = model.predict(X_test)

122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


In [14]:
poll = np.array(df["no2"])

meanop = poll.mean()
stdop = poll.std()

y_test_true = y_test*stdop + meanop
testPredict = forecast*stdop + meanop

In [15]:
from sklearn.metrics import mean_squared_error

In [16]:
rmse = np.sqrt(mean_squared_error(y_test_true, testPredict))
print("Test RMSE ="  ,rmse)

Test RMSE = 2.1713424


In [20]:
# Calculate MAPE
mape = np.mean(np.abs((y_test_true - testPredict) / y_test_true)) * 100
print("Test MAPE =", mape)

Test MAPE = 4.979865252971649


In [21]:
# Calculate accuracy as 1 - MAPE
accuracy = 100 - mape
print("Accuracy =", accuracy, "%")

Accuracy = 95.02013474702835 %


In [26]:
from sklearn.metrics import mean_absolute_error

# Calculate MAE
mae = mean_absolute_error(y_test_true, testPredict)
print("Test MAE =", mae)

Test MAE = 1.4916686


In [22]:
forecast

array([[ 0.16981833,  0.15109107,  0.1378942 , ...,  0.2358299 ,
         0.2117889 ,  0.18536675],
       [ 0.00861526,  0.00597645, -0.01023901, ...,  0.0202971 ,
         0.00058155, -0.00792895],
       [ 0.06256454,  0.04195713,  0.01659228, ...,  0.09612297,
         0.09316175,  0.09619264],
       ...,
       [ 0.04371309,  0.14400971,  0.25576007, ...,  0.11368106,
         0.10502083,  0.07754014],
       [ 0.49073797,  0.4287088 ,  0.3246286 , ...,  0.30290583,
         0.2316498 ,  0.24574539],
       [ 0.1947615 ,  0.20698282,  0.2127686 , ...,  0.10175429,
         0.14002061,  0.18895845]], dtype=float32)

In [23]:
testPredict

array([[28.390282, 27.995417, 27.717161, ..., 29.782133, 29.275229,
        28.718119],
       [24.99132 , 24.93568 , 24.593779, ..., 25.23763 , 24.82193 ,
        24.642487],
       [26.12884 , 25.694332, 25.159515, ..., 26.836416, 26.77398 ,
        26.837887],
       ...,
       [25.731356, 27.846107, 30.20236 , ..., 27.206629, 27.024029,
        26.4446  ],
       [35.15686 , 33.848976, 31.65445 , ..., 31.196426, 29.693996,
        29.9912  ],
       [28.916206, 29.173893, 29.295887, ..., 26.955153, 27.761997,
        28.79385 ]], dtype=float32)

In [24]:
y_test_true

array([[27.915205, 28.454588, 29.010315, ..., 30.530396, 29.549698,
        28.569002],
       [25.136566, 25.25098 , 25.34905 , ..., 25.463465, 25.34905 ,
        25.25098 ],
       [26.002848, 25.577879, 25.136566, ..., 26.722025, 26.493196,
        26.280712],
       ...,
       [25.626915, 28.078655, 30.530396, ..., 28.19307 , 27.392168,
        26.60761 ],
       [35.15601 , 34.616627, 33.145584, ..., 35.384842, 35.106976,
        34.894493],
       [28.8959  , 29.059351, 29.2228  , ..., 27.048923, 28.454588,
        29.876596]], dtype=float32)

In [44]:
import numpy as np

def calculate_segment_rmse(predictions, actuals, segment_length=1):
    num_segments = predictions.shape[1] // segment_length
    rmses = []
    
    for i in range(num_segments):
        start_idx = i * segment_length
        end_idx = start_idx + segment_length
        
        # Extract the segment for both predictions and actuals
        pred_segment = predictions[:, start_idx:end_idx]
        actual_segment = actuals[:, start_idx:end_idx]
        
        # Calculate RMSE for this segment
        segment_rmse = np.sqrt(mean_squared_error(actual_segment.flatten(), pred_segment.flatten()))
        rmses.append(segment_rmse)
    
    return rmses

In [45]:
segment_rmses = calculate_segment_rmse(testPredict, y_test_true)
for i, rmse in enumerate(segment_rmses):
    print(f"RMSE for segment {i + 1}: {rmse:.4f}")

RMSE for segment 1: 0.5052
RMSE for segment 2: 0.9399
RMSE for segment 3: 1.3419
RMSE for segment 4: 1.5356
RMSE for segment 5: 1.6742
RMSE for segment 6: 1.7868
RMSE for segment 7: 1.8815
RMSE for segment 8: 1.9198
RMSE for segment 9: 1.9538
RMSE for segment 10: 1.9706
RMSE for segment 11: 1.9612
RMSE for segment 12: 1.9639
RMSE for segment 13: 1.9771
RMSE for segment 14: 1.9490
RMSE for segment 15: 1.9426
RMSE for segment 16: 1.9253
RMSE for segment 17: 1.9269
RMSE for segment 18: 1.9397
RMSE for segment 19: 1.9419
RMSE for segment 20: 1.9415
RMSE for segment 21: 1.9657
RMSE for segment 22: 1.9762
RMSE for segment 23: 1.9708
RMSE for segment 24: 1.9783
RMSE for segment 25: 1.9872
RMSE for segment 26: 1.9972
RMSE for segment 27: 2.0359
RMSE for segment 28: 2.0612
RMSE for segment 29: 2.0946
RMSE for segment 30: 2.1333
RMSE for segment 31: 2.1484
RMSE for segment 32: 2.1579
RMSE for segment 33: 2.1705
RMSE for segment 34: 2.1694
RMSE for segment 35: 2.1604
RMSE for segment 36: 2.1744
R

In [46]:
model.save("no2_1dcnn.keras")